In [15]:
import os
import pandas as pd
import google.generativeai as genai

from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

generation_config = {
    "temperature": 1,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

safety_settings = [
    {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_ONLY_HIGH"
    },
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "threshold": "BLOCK_ONLY_HIGH"
    },
    {
      "category": "HARM_CATEGORY_HARASSMENT",
      "threshold": "BLOCK_ONLY_HIGH"
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "threshold": "BLOCK_ONLY_HIGH"
    },
]

model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    generation_config=generation_config,
    system_instruction="Humanize image prompts\nYour task is to convert a highly complex and technical image into a layman's prompt. A much shorter, simpler, non technical details but can be inferred upon only.",
    safety_settings=safety_settings
)

def convert_prompts(advanced_prompt):
    chat_session = model.start_chat(history=[
        {
            "role": "user",
            "parts": [advanced_prompt],
        }
    ])
    response = chat_session.send_message(advanced_prompt)
    return response.text

In [16]:
import pandas as pd

df = pd.read_excel('test.xlsx', header=None)

human_prompts = []
for advanced_prompt in df.iloc[:, 0]:
    human_prompts.append(convert_prompts(advanced_prompt))

results_df = pd.DataFrame({'advanced_prompts': df.iloc[:, 0], 'human_prompts': human_prompts})

results_df.to_excel('output.xlsx', index=False)
